<a href="https://colab.research.google.com/github/NataliaPoluektova/TPBA/blob/main/prakt7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практична робота №7
**Тема:** Підготовка даних для аналізу

## Мета роботи
1. Навчитися очищувати, об'єднувати, перегортати, дискретизувати дані перед аналізом.

Для використання DataFrame потрібно імпортувати бібліотеку pandas:

In [ ]:
import pandas as pd

**Завдання 1.  Очищення даних про продажі**
Є файл Sales_q з колонками: Рік, Квартал, Товар, Продажі

Переглянути його, знайти проблеми та виправити(Пропущене значення у колонці "Продажі" (Q3 2023),Підозрілий викид: 300000 у Q3 2024)

In [19]:
# Завантаження даних
df = pd.read_csv("sales_q.csv")
print(df)
# 1. Перевірка пропущених значень
print("Пропущені значення у 'Продажі':", df["Продажі"].isna().sum())

# 2. Обчислення середнього значення (без пропусків)
mean_sales = df["Продажі"].mean(skipna=True)
print(f"Середнє значення продажів: {mean_sales:.2f}")

# 3. Заповнення пропусків середнім
df["Продажі"] = df["Продажі"].fillna(mean_sales)

# 4. Визначення порогу для викидів (+100% від середнього)
threshold = mean_sales * 2
df["Викид"] = df["Продажі"] > threshold

# 5. Збереження очищеного файлу
df.to_csv("sales_q_cleaned.csv", index=False)

# 6. Виведення підозрілих кварталів
print("Квартали з аномально високими продажами:")
print(df[df["Викид"]][["Рік", "Квартал", "Продажі"]])


    Рік Квартал  Товар   Продажі
0  2023      Q1  Latte   12000.0
1  2023      Q2  Latte   13500.0
2  2023      Q3  Latte       NaN
3  2023      Q4  Latte   16000.0
4  2024      Q1  Latte   17000.0
5  2024      Q2  Latte   18500.0
6  2024      Q3  Latte  300000.0
7  2024      Q4  Latte   19000.0
Пропущені значення у 'Продажі': 1
Середнє значення продажів: 56571.43
Квартали з аномально високими продажами:
    Рік Квартал   Продажі
6  2024      Q3  300000.0


**Завдання для самостійного виконання:**

Зробіть таке ж очищення для даних з файлу clients.csv (Пропущене значення у "Вік" (Марія у 2024), Пропущене значення у "Місто" (Андрій у 2024))

In [ ]:
# Ваш код тут

**Задача 2: Об’єднання даних про клієнтів і продажі"**
Спільні ключі: Рік, Квартал Мета: Зіставити клієнтів з квартальними продажами, проаналізувати активність по містах

Об’єднання даних дозволяє:

зіставити поведінку клієнтів з фінансовими результатами

виявити, які міста або вікові групи пов’язані з високими продажами

створити основу для сегментованої аналітики та таргетованих рішень

In [21]:

# Завантаження даних
df_sales = pd.read_csv("sales_q.csv")
df_clients = pd.read_csv("clients.csv")

# Очищення пропусків у продажах
df_sales["Продажі"] = df_sales["Продажі"].fillna(df_sales["Продажі"].mean())

# Об'єднання по Рік і Квартал
merged = pd.merge(df_clients, df_sales, on=["Рік", "Квартал"], how="left")

# Групування: середні продажі по містах
grouped = merged.groupby("Місто")["Продажі"].mean().sort_values(ascending=False)

print("Середні продажі по містах:")
print(grouped)

Середні продажі по містах:
Місто
Київ      15250.0
Львів     15250.0
Одеса     15250.0
Харків    12000.0
Name: Продажі, dtype: float64


**Завдання для самостійного виконання:** Аналіз по кварталах

Файли: sales_q.csv + clients.csv

Об’єднайте таблиці по "Рік" і "Квартал" (тип left).

Порахуйте середні і мвксимальні продажі по кожному кварталу

Виведіть результат у вигляді таблиці.


In [ ]:
#Ваш код тут:

Ваши висновки тут:

**Задача 3: Дискретизація віку клієнтів**

Розбити клієнтів на вікові групи для подальшого аналізу

Дискретизація (або бінінг) — це розбиття числових значень на категорії. У бізнес-аналітиці це дозволяє:

сегментувати клієнтів за віком, доходом, витратами

будувати таргетовані пропозиції

виявляти закономірності в поведінці груп

У pandas це робиться через pd.cut() або pd.qcut().

In [22]:
# Завантаження даних
df = pd.read_csv("clients.csv")

# Заповнення пропусків у віці
df["Вік"] = df["Вік"].fillna(df["Вік"].mean())

# Визначення вікових груп
bins = [0, 30, 45, 100]
labels = ["<30", "30–45", "45+"]

# Дискретизація
df["Вікова група"] = pd.cut(df["Вік"], bins=bins, labels=labels)

# Виведення кількості клієнтів у кожній групі
print("Розподіл клієнтів за віком:")
print(df["Вікова група"].value_counts())

Розподіл клієнтів за віком:
Вікова група
<30      4
30–45    4
45+      0
Name: count, dtype: int64


**Завдання для самостійного виконання:** Для задачі 3 порахуйте середні продажі для кожної вікової групи. Яка група найактивніша?

Виведіть результат у вигляді таблиці.

In [ ]:
#Ваш код тут:

Ваши висновки тут:

**Задача 4: Перегрупування по ієрархічних індексах**
Файл: sales_q.csv
Колонки: Рік, Квартал, Товар, Продажі Мета: Створити багаторівневу структуру для гнучкого аналізу

Виконайте код нижче:

In [23]:
# Завантаження даних
df = pd.read_csv("sales_q.csv")

# Заповнення пропусків
df["Продажі"] = df["Продажі"].fillna(df["Продажі"].mean())

# Групування по Рік і Квартал
grouped = df.groupby(["Рік", "Квартал"]).agg({"Продажі": "sum"})

# Перетворення в ієрархічний індекс
grouped.index = pd.MultiIndex.from_tuples(grouped.index, names=["Рік", "Квартал"])

# Виведення результату
print("Продажі по роках і кварталах:")
print(grouped)

Продажі по роках і кварталах:
                    Продажі
Рік  Квартал               
2023 Q1        12000.000000
     Q2        13500.000000
     Q3        56571.428571
     Q4        16000.000000
2024 Q1        17000.000000
     Q2        18500.000000
     Q3       300000.000000
     Q4        19000.000000


**Завдання для самостійного виконання:**

Перегрупування клієнтів
Файл: clients.csv

Створіть групування по "Місто" і "Рік".

Порахуйте кількість клієнтів у кожному місті за кожен рік.

Перетворіть індекс на ієрархічний (Місто, Рік).

Виведіть таблицю, де видно активність по містах у 2023 і 2024.

In [ ]:
# Ваш код тут

Ваші висновки тут:

**Задача 5: Динаміка витрат**
Обчисліть абсолютну та відносну зміну витрат між місяцями.

Виконайте код нижче:

In [ ]:
df["Δ витрат"] = df["Витрати"].diff()
df["% зміна витрат"] = df["Витрати"].pct_change().round(3)
print(df[["Витрати", "Δ витрат", "% зміна витрат"]])

**Завдання для самостійного виконання:** Обчисліть зміну доходів (diff і pct_change). У яких місяцях доходи зросли понад 5%?

In [ ]:
# Ваш код тут

Ваши висновки тут:

**Індивідуальне завдання**
є однакові для всіх дані:

```
df = pd.DataFrame({
    "Регіон": ["Київ", "Львів", "Одеса", "Харків", "Дніпро", "Запоріжжя"],
    "Витрати": [12000, 13500, 11000, 14000, 12500, 10000],
    "Доходи": [15000, 16000, 14500, 17000, 15500, 13000],
    "Населення": [2800, 2500, 1000, 1500, 1200, 700]
```
Розв'яжіть задачу свого варіанту.

🔹 Варіант 1
Обчисліть прибуток по кожному регіону. Додайте стовпець "Податок" (20% від прибутку). Який регіон має найбільший податок?

🔹 Варіант 2
Обчисліть витрати на одну особу в кожному регіоні. Який регіон витрачає найменше на одну людину?

🔹 Варіант 3
Обчисліть прибуток і додайте стовпець "Чистий прибуток" = "Прибуток" – "Податок". Відсортуйте таблицю за чистим прибутком у спадному порядку.

🔹 Варіант 4
Знайдіть регіони, де прибуток перевищує 3000 грн. Які з них мають витрати нижче середнього?

🔹 Варіант 5
Обчисліть середні витрати та доходи по всіх регіонах. Які регіони мають витрати вище середнього, але прибуток нижче середнього?

🔹 Варіант 6
Обчисліть прибуток. Виведіть таблицю, відсортовану за витратами у зростаючому порядку. Який регіон має найменші витрати, але не найвищий прибуток?

🔹 Варіант 7
Обчисліть прибуток. Додайте стовпець "Рентабельність" = прибуток / витрати. Який регіон має найвищу рентабельність?

🔹 Варіант 8
Обчисліть прибуток. Знайдіть регіони, де податок перевищує 600 грн. Який з них має найменше населення?

🔹 Варіант 9
Обчисліть прибуток. Побудуйте короткий статистичний опис прибутку. Які регіони мають прибуток нижче медіани?

🔹 Варіант 10
Обчисліть прибуток. Визначте, у яких регіонах витрати перевищують 13000 грн, але прибуток не перевищує 3000 грн. Які висновки можна зробити?

In [ ]:
#Ваш код тут:

Ваші висновки тут: